In [30]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dizionari

data = pd.read_csv("../data/ISTAT_Italian_Excursion_2014_2022.csv")


In [31]:
# Inizializzazione dell'app Dash con il tema Material
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])

radio_linechart = dbc.RadioItems(
    id="radio-linecharts-id",
    options=[
        {"label": "Età", "value": "eta10"},
        {"label": "Estero", "value": "DEST_ERE"},
    ],
    value="eta10",
    inline=True
)



hist_feature_dropdown = dcc.Dropdown(
    id='hist-feature',
    options=[{'label': dizionari.dizionario_nomi[col], 'value': col} for col in data.columns],
    value='mese'  # Valore predefinito
)

violin_feature_dropdown = dcc.Dropdown(
    id='violin-feature',
    options=[{'label': dizionari.dizionario_nomi[col], 'value': col} for col in data.columns],
    value='mese'  # Valore predefinito
)

variabili_checklist = dbc.Checklist(
    id='correlation-checklist',
    options=[{'label': dizionari.dizionario_nomi[col], 'value': col} for col in data.columns],
    value=['mese', 'DEST_ERE', 'DEST_EIE'],  # Valori predefiniti
    inline=True
)

# Layout dell'app
app.layout = dbc.Container([
    html.H1("Analisi escursioni Italia"),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              radio_linechart,
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              hist_feature_dropdown,
            ])
        , width=6, align="center"),
    ]),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='graph')
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='histogram')
            ])
        , width=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                html.Label("Select a feature to visualize with a violin plot:"),
                violin_feature_dropdown,
            ])
        ], width=4, align="center"),
        dbc.Col([
            dbc.Stack([
              html.Label("Seleziona le variabili per la heatmap di correlazione:"),
              variabili_checklist,
            ])
        ], width=8, align="center"),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                dcc.Graph(id='violin-plot')
            ])
        ], width=4),
        dbc.Col([
            dbc.Stack([
              dcc.Graph(id='heatmap')
            ])
        ], width=8),
    ]),
])

# LINECHART - Evoluzione degli spostami negli anni (suddivisi per età)
# Callback per il grafico a dispersione (scatter plot)
@app.callback(
    Output('graph', 'figure'),
    [Input('radio-linecharts-id', 'value')]
)
def update_line_chart(radio_linechart):
    if radio_linechart == "eta10":
        data['Età'] = data['eta10'].map(dizionari.eta_mapping)
        conteggio = data.groupby(['annrif', 'eta10']).size().reset_index(name='count')
    elif radio_linechart == "DEST_ERE":
        conteggio = data.groupby(['annrif', 'DEST_ERE']).size().reset_index(name='count')
        conteggio = conteggio[conteggio['DEST_ERE'] > 100]

    colori = ['#FF0000', '#FF4500', '#FF8C00', '#FFD700', '#ADFF2F', '#7FFF00', '#32CD32', '#008000']

    fig = px.line(conteggio,
            x='annrif',
            y='count',
            color=radio_linechart,
            title='Evoluzione degli spostamenti negli anni (Età)',
            color_discrete_sequence=colori)

    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50))


# fig = px.scatter(boston_data, x=xaxis_feature, y=yaxis_feature, color='mese',
#                    title=f'Grafico a dispersione di {dizionari.dizionario_nomi[yaxis_feature]} vs {dizionari.dizionario_nomi[xaxis_feature]}',
#                    labels={xaxis_feature: xaxis_feature, yaxis_feature: yaxis_feature})

    return fig

# Esecuzione dell'app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")





Dash app running on http://127.0.0.1:8050/
